In [18]:
!pip install geopandas shapely tqdm

Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:833)'),)) - skipping


In [25]:
import geopandas as gpd
from shapely.geometry import Point
from scipy.spatial import cKDTree
import numpy as np
from tqdm import tqdm
import pandas as pd
import numpy as np

# Load shp file
gdf_origin = gpd.read_file("E:/Users/Administrator/Downloads/疫苗公平性/new_orgin_169.shp")
gdf_dest = gpd.read_file("E:/Users/Administrator/Downloads/疫苗公平性/new_vac.shp")


In [26]:
gdf_origin

,pointid,grid_code,Object,geometry
0,6223,18.8979,6126.0,POINT (172.97000 -34.64500)
1,7888,30.7599,7791.0,POINT (173.08000 -34.77333)
2,8008,23.3576,7911.0,POINT (173.09000 -34.78083)
3,8283,10.2848,8175.0,POINT (173.11667 -34.81083)
4,8381,10.2018,8268.0,POINT (173.12583 -34.82167)
...,...,...,...,...
169798,400924,12.6874,383810.0,POINT (168.12167 -46.90000)
169799,400925,12.7887,383811.0,POINT (168.12250 -46.90000)
169800,400931,11.9452,383816.0,POINT (168.12417 -46.90083)
169801,400936,11.1636,383821.0,POINT (168.12583 -46.90167)


In [27]:
# Set the coordinate reference system (CRS) to WGS84
gdf_origin.set_crs("EPSG:4326", inplace=True)
gdf_dest.set_crs("EPSG:4326", inplace=True)

# Convert CRS to UTM projected coordinates (select according to region)
gdf_origin = gdf_origin.to_crs("EPSG:32760")
gdf_dest = gdf_dest.to_crs("EPSG:32760")

origin_points = gdf_origin.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()
dest_points = gdf_dest.geometry.apply(lambda geom: (geom.x, geom.y)).tolist()

In [30]:
# Build KDTree
tree = cKDTree(dest_points)

# Calculate the distance and index of the nearest 3 points
distances, indices = tree.query(origin_points, k=3)

gdf_result = gpd.GeoDataFrame(columns=['origin', 'dest', 'distance'], crs="EPSG:32760")
for i in tqdm(range(len(distances))):
    # For each origin point, obtain the coordinates and distances of the three nearest target points
    for j in range(3):
        dest_point = dest_points[indices[i][j]]
        distance = distances[i][j] / 1000  # Convert distance units to kilometers
        gdf_result = gdf_result.append({
            'origin': Point(origin_points[i]),
            'dest': Point(dest_point),
            'distance': distance
        }, ignore_index=True)

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:8: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.
  
100%|████████████████████████████████████████████████████████████████████████| 169803/169803 [1:58:10<00:00, 23.95it/s]


In [31]:
gdf_result.to_csv("E:/Users/Administrator/Downloads/疫苗公平性/result_filter10.csv")

In [61]:
!pip install --upgrade pyproj

ERROR: Could not install packages due to an OSError: Could not find a suitable TLS CA certificate bundle, invalid path: C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\certifi\cacert.pem



In [69]:
# Extract the origin column of gdf_origin
gdf_origin_sub = gdf_origin[['origin']]
# Extract the dest and distance columns of gdf_dest
gdf_dest_sub = gdf_dest[['dest', 'distance']]

merged_gdf = pd.merge(gdf_origin_sub, gdf_dest_sub, left_index=True, right_index=True)

In [108]:
import pandas as pd
import requests
import json
from tqdm import tqdm

# OSRM API URL
osrm_url = "http://localhost:9000/route/v1/foot/"

results_df = pd.DataFrame(columns=["origin", "dest", "distance", "duration"])

for _, row in tqdm(merged_gdf1.iterrows(), total=merged_gdf1.shape[0]):
    origin = [row['origin'].x, row['origin'].y]
    dest = [row['dest'].x, row['dest'].y]
    location_pair = [origin, dest]
    location_str = ';'.join([f"{lon},{lat}" for lon, lat in location_pair])
    url = osrm_url + location_str
    response = requests.get(url)
    data = response.json()

    try:
        for route in data['routes']:
            results_df3 = results_df3.append({
                "origin": origin,
                "dest": dest,
                "distance": route['distance'],
                "duration": route['duration']
            }, ignore_index=True)
    except KeyError:
        # If there is no 'routes' key, continue with the next iteration
        results_df3 = results_df3.append({
                "origin": origin,
                "dest": dest,
                "distance": None,
                "duration": None
            }, ignore_index=True)
        continue

100%|████████████████████████████████████████████████████████████████████████████| 42798/42798 [08:51<00:00, 80.45it/s]


In [110]:
results_df3.to_csv("E:/Users/Administrator/Downloads/疫苗公平性/route_466611后.csv")

In [126]:
# Convert the origin column into string format for deduplication
df['origin'] = df['origin'].astype(str)

# The origin is the same, keep the row with the smallest distance
df_min_distance = df.loc[df.groupby('origin')['distance'].idxmin()]


In [127]:
df_min_distance

,Unnamed: 0,origin,dest,distance,duration
481608,481680,"[167.6149993447974, -45.5641662830277]","[168.01362170000004, -46.351761299999964]",126782.1,7161.9
481611,481683,"[167.6158326781274, -45.5641662830277]","[168.01362170000004, -46.351761299999964]",126712.7,7155.6
481614,481686,"[167.61666601145726, -45.564166283027696]","[168.01362170000004, -46.351761299999964]",127021.5,7205.9
499602,499674,"[167.66166601127736, -46.19333294716591]","[168.01362170000004, -46.351761299999964]",40178.8,1867.3
499605,499677,"[167.66249934460734, -46.19333294716591]","[168.01362170000004, -46.351761299999964]",40193.4,1868.0
...,...,...,...,...,...
227043,227043,"[178.41249930161473, -37.79249964759395]","[178.31943890000002, -37.88995259999996]",21629.3,1358.0
227394,227394,"[178.41249930161473, -37.79333298092393]","[178.31943890000002, -37.88995259999996]",21499.3,1339.2
190305,190305,"[178.42333263490468, -37.63916631487683]","[178.31943890000002, -37.88995259999996]",51065.9,3260.9
190266,190266,"[178.4441659681545, -37.638332981546846]","[178.31943890000002, -37.88995259999996]",52681.1,3493.1


In [132]:
# Split the origin and dest columns
df_min_distance[['origin_x', 'origin_y']] = pd.DataFrame(df_min_distance['origin'].to_list(), index=df_min_distance.index)
df_min_distance[['dest_x', 'dest_y']] = pd.DataFrame(df_min_distance['dest'].to_list(), index=df_min_distance.index)


In [134]:
df_min_distance.to_csv("E:/Users/Administrator/Downloads/疫苗公平性/NewZealand_calculate.csv")